In [6]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from preprocessing import (
    clsf_boz_cd_c, clsf_crd_grd, get_disc_factor_all_c, get_loss_adj_rate_all_c
)

In [ ]:
#TODO : drop -> filter로 변경

In [7]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format
os.makedirs('result', exist_ok=True)

# 전역변수
FILE_PATH = Path('./data/현행추정부채_자동차')
BASE_YYMM = '201912'

In [16]:
# 데이터 불러오기
## 보험료부채 익스포져
자동차_원수_미경과보험료 = pd.read_excel(FILE_PATH / f'자동차_원수_미경과보험료_{BASE_YYMM}.xlsx', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'USTK_DVCD': str})
자동차_비례출재_미경과보험료 = pd.read_excel(FILE_PATH / f'자동차_비례출재_미경과보험료_{BASE_YYMM}.xlsx', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'USTK_DVCD': str})
자동차_비비례출재_미경과보험료 = pd.read_excel(FILE_PATH / f'자동차_비비례출재_미경과보험료_{BASE_YYMM}.xlsx', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'SL_CVR_CD': str, 'T02_RN_RINSC_CD': str})
# 일반_원수_예정보험료 = pd.read_excel(FILE_PATH / f'일반_원수_예정보험료_{BASE_YYMM}.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str})
# 일반_출재_예정보험료 = pd.read_excel(FILE_PATH / f'일반_출재_예정보험료_{BASE_YYMM}.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str, 'T02_RN_RINSC_CD': str})
자동차_원수_직전3년보험금손조비 = pd.read_excel(FILE_PATH / f'자동차_원수_직전3년보험금손조비_{BASE_YYMM}.xlsx', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str})
자동차_비례출재_직전3년보험금손조비 = pd.read_excel(FILE_PATH / f'자동차_비례출재_직전3년보험금손조비_{BASE_YYMM}.xlsx', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str})
자동차_비비례출재_직전3년보험금손조비 = pd.read_excel(FILE_PATH / f'자동차_비비례출재_직전3년보험금손조비_{BASE_YYMM}.xlsx', dtype={'PDC_CD': str, 'T02_RN_RINSC_CD': str})

## 준비금부채 익스포져
자동차_원수_개별추산액 = pd.read_excel(FILE_PATH / f'자동차_원수_개별추산액_{BASE_YYMM}.xlsx', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str})
자동차_비례출재_개별추산액 = pd.read_excel(FILE_PATH / f'자동차_비례출재_개별추산액_{BASE_YYMM}.xlsx', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str})
자동차_비비례출재_개별추산액 = pd.read_excel(FILE_PATH / f'자동차_비비례출재_개별추산액_{BASE_YYMM}.xlsx', dtype={'T02_RN_RINSC_CD': str, 'PDC_CD': str})

## 기초정보, 가정 등
# 자동차_코드맵핑 = pd.read_excel(FILE_PATH / '자동차_코드맵핑.xlsx', 
#     dtype={'PDGR_CD': str, 'BSC_CVR_CD': str, 'INER_CHN_DVCD': str, 'USTK_DVCD': str, 'AT_PDGR_CD': str, 'AT_BSC_CVR_CD': str, 'AT_CHN_DVCD': str})
# 자동차_코드맵핑.to_csv(FILE_PATH / '자동차_코드맵핑.csv', index=False)
자동차_코드맵핑 = pd.read_csv(FILE_PATH / '자동차_코드맵핑.csv',
    dtype={'PDC_CD': str, 'PDGR_CD': str, 'BSC_CVR_CD': str, 'INER_CHN_DVCD': str, 'USTK_DVCD': str, 'AT_PDGR_CD': str, 'AT_BSC_CVR_CD': str, 'AT_CHN_DVCD': str})
자동차_상품군정보 = pd.read_excel(FILE_PATH / '자동차_상품군정보.xlsx', dtype={'PDGR_CD': str, 'BSC_CVR_CD': str})
자동차_보험금진전추이 = pd.read_excel(FILE_PATH / f'자동차_보험금진전추이_{201812}.xlsx', dtype={'PDGR_CD': str, 'AY': str}) \
        .rename(columns={'AT_PDGR_CD': 'BOZ_CD', 'AY_YM': 'AY'})
자동차_최종손해율 = pd.read_excel(FILE_PATH / f'자동차_최종손해율_{201812}.xlsx', dtype={'RRNR_DVCD': str, 'AT_CHN_DVCD': str, 'AT_PDGR_CD': str}) \
    .drop('RRNR_DAT_DVCD', axis=1)
자동차_사업비율 = pd.read_excel(FILE_PATH / f'자동차_사업비율_{BASE_YYMM}.xlsx', dtype={'RRNR_DVCD': str, 'BOZ_CD': str})
자동차_IBNR = pd.read_excel(FILE_PATH / f'자동차_IBNR_{BASE_YYMM}.xlsx', dtype={'RRNR_DVCD': str, 'BSC_CVR_CD': str})

## 공통
재보험자_국내신용등급 = pd.read_excel(FILE_PATH / f'재보험자_국내신용등급_{202012}.xlsx', dtype={'재보험사코드': str}) \
        .rename(columns = {'재보험사코드': 'T02_RN_RINSC_CD', '국내신용등급': 'CRD_GRD'})
선도부도율 = pd.read_excel(FILE_PATH / f'선도부도율_{202012}.xlsx')
할인율 = pd.read_excel(FILE_PATH / f'할인율_{202012}.xlsx')

In [122]:
자동차_원수_미경과보험료_가공

,BSC_CVR_CD,PDGR_CD,PDC_CD,INER_CHN_DVCD,USTK_DVCD,LTPD_URND_PRM,AT_PDGR_CD,AT_BSC_CVR_CD,AT_CHN_DVCD,RRNR_DVCD,DMFR_DVCD,CRD_GRD,BOZ_CD
0,09,09,20055,14,1,863439937,02,09,04,01,01,#,B004
1,01,09,20023,01,1,16029630462,02,01,03,01,01,#,B003
2,02,09,20016,99,1,18762792018,02,01,01,01,01,#,B003
3,01,08,20014,01,1,66398035275,01,01,03,01,01,#,B001
4,09,08,20001,04,1,14231290541,01,09,02,01,01,#,B002
...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,03,19,20040,10,2,1200700,11,03,01,01,01,#,B002
462,01,21,20042,11,2,13363,11,01,01,01,01,#,B005
463,02,10,20058,10,1,471772,03,01,01,01,01,#,B005
464,02,20,20041,10,2,462683,11,01,01,01,01,#,B003


In [127]:
자동차_비비례출재_미경과보험료_가공

,BSC_CVR_CD,PDGR_CD,PDC_CD,SL_CVR_CD,T02_RN_RINSC_CD,LTPD_URND_PRM,RRNR_DVCD,DMFR_DVCD,BOZ_CD,CRD_GRD
0,02,,,200002,185055,0,03,01,B007,AA-
1,06,,,200008,124012,0,03,01,B007,AA
2,02,,,200002,185083,0,03,01,B007,무등급
3,02,,,200002,185097,0,03,01,B007,AA+
4,02,,,200002,124012,0,03,01,B007,AA
5,02,,,200002,122053,0,03,01,B007,AA


In [128]:
자동차_비비례출재_미경과보험료_가공

,BSC_CVR_CD,PDGR_CD,PDC_CD,SL_CVR_CD,T02_RN_RINSC_CD,LTPD_URND_PRM,RRNR_DVCD,DMFR_DVCD,BOZ_CD,CRD_GRD
0,02,,,200002,185055,0,03,01,B007,AA-
1,06,,,200008,124012,0,03,01,B007,AA
2,02,,,200002,185083,0,03,01,B007,무등급
3,02,,,200002,185097,0,03,01,B007,AA+
4,02,,,200002,124012,0,03,01,B007,AA
5,02,,,200002,122053,0,03,01,B007,AA


In [148]:
# 데이터 전처리
## 미경과보험료
자동차_원수_미경과보험료_가공 = 자동차_원수_미경과보험료.copy()
자동차_원수_미경과보험료_가공 = 자동차_원수_미경과보험료_가공.merge(자동차_코드맵핑, on=['PDC_CD', 'PDGR_CD', 'BSC_CVR_CD', 'INER_CHN_DVCD', 'USTK_DVCD'], how='left')
자동차_원수_미경과보험료_가공['RRNR_DVCD'] = '01'
자동차_원수_미경과보험료_가공['DMFR_DVCD'] = '01'
자동차_원수_미경과보험료_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_원수_미경과보험료_가공, 자동차_상품군정보)
자동차_원수_미경과보험료_가공['CRD_GRD'] = '#'
자동차_원수_미경과보험료_가공.rename(columns={'LTPD_URND_PRM': 'URND_PRM'}, inplace=True)
자동차_원수_미경과보험료_가공 = 자동차_원수_미경과보험료_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'AT_PDGR_CD', 'AT_BSC_CVR_CD', 'CRD_GRD', 'URND_PRM']]

자동차_비례출재_미경과보험료_가공 = 자동차_비례출재_미경과보험료.copy()
자동차_비례출재_미경과보험료_가공 = 자동차_비례출재_미경과보험료_가공.merge(자동차_코드맵핑, on=['PDC_CD', 'PDGR_CD', 'BSC_CVR_CD', 'INER_CHN_DVCD', 'USTK_DVCD'], how='left')
자동차_비례출재_미경과보험료_가공['RRNR_DVCD'] = '03'
자동차_비례출재_미경과보험료_가공['DMFR_DVCD'] = '01'
자동차_비례출재_미경과보험료_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_비례출재_미경과보험료_가공, 자동차_상품군정보)
자동차_비례출재_미경과보험료_가공['T02_RN_RINSC_CD'] = '124012'
자동차_비례출재_미경과보험료_가공['CRD_GRD'] = clsf_crd_grd(자동차_비례출재_미경과보험료_가공, 재보험자_국내신용등급)
자동차_비례출재_미경과보험료_가공.rename(columns={'RN_LTPD_URND_PRM': 'URND_PRM'}, inplace=True)
자동차_비례출재_미경과보험료_가공 = 자동차_비례출재_미경과보험료_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'AT_PDGR_CD', 'AT_BSC_CVR_CD', 'CRD_GRD', 'URND_PRM']]

자동차_비비례출재_미경과보험료_가공 = 자동차_비비례출재_미경과보험료.copy()
자동차_비비례출재_미경과보험료_가공['RRNR_DVCD'] = '03'
자동차_비비례출재_미경과보험료_가공['DMFR_DVCD'] = '01'
자동차_비비례출재_미경과보험료_가공['BOZ_CD'] = 'B007' # 수정필요
자동차_비비례출재_미경과보험료_가공['CRD_GRD'] = clsf_crd_grd(자동차_비비례출재_미경과보험료_가공, 재보험자_국내신용등급)
자동차_비비례출재_미경과보험료_가공['AT_PDGR_CD'] = '01' # 수정필요 (현재는 미경과보험료 0으로 영향은 없음)
자동차_비비례출재_미경과보험료_가공['AT_BSC_CVR_CD'] = '01' # 수정필요 (현재는 미경과보험료 0으로 영향은 없음)
자동차_비비례출재_미경과보험료_가공.rename(columns={'LTPD_URND_PRM': 'URND_PRM'}, inplace=True)
자동차_비비례출재_미경과보험료_가공 = 자동차_비비례출재_미경과보험료_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'AT_PDGR_CD', 'AT_BSC_CVR_CD', 'CRD_GRD', 'URND_PRM']]

자동차_미경과보험료_가공 = pd.concat([자동차_원수_미경과보험료_가공, 자동차_비례출재_미경과보험료_가공, 자동차_비비례출재_미경과보험료_가공], axis=0).reset_index(drop=True)

## 직전3년보험금손조비
자동차_원수_직전3년보험금손조비_가공 = 자동차_원수_직전3년보험금손조비.copy()
자동차_원수_직전3년보험금손조비_가공['RRNR_DVCD'] = '01'
자동차_원수_직전3년보험금손조비_가공['DMFR_DVCD'] = '01'
자동차_원수_직전3년보험금손조비_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_원수_직전3년보험금손조비_가공, 자동차_상품군정보)
자동차_원수_직전3년보험금손조비_가공['CRD_GRD'] = '#'
자동차_원수_직전3년보험금손조비_가공.rename(columns={'AT_DAG_IVMT': 'DAG_IVMT'}, inplace=True)
자동차_원수_직전3년보험금손조비_가공 = 자동차_원수_직전3년보험금손조비_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'CRD_GRD', 'PYN_BNF', 'DAG_IVMT']]

자동차_비례출재_직전3년보험금손조비_가공 = 자동차_비례출재_직전3년보험금손조비.copy()
자동차_비례출재_직전3년보험금손조비_가공 = 자동차_비례출재_직전3년보험금손조비_가공.assign(RN_DAG_IVMT=lambda x: x['RN_DAG_IVMT'].fillna(0))
자동차_비례출재_직전3년보험금손조비_가공['RRNR_DVCD'] = '03'
자동차_비례출재_직전3년보험금손조비_가공['DMFR_DVCD'] = '01'
자동차_비례출재_직전3년보험금손조비_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_비례출재_직전3년보험금손조비_가공, 자동차_상품군정보)
자동차_비례출재_직전3년보험금손조비_가공['T02_RN_RINSC_CD'] = '124012'
자동차_비례출재_직전3년보험금손조비_가공['CRD_GRD'] = clsf_crd_grd(자동차_비례출재_직전3년보험금손조비_가공, 재보험자_국내신용등급)
자동차_비례출재_직전3년보험금손조비_가공.rename(columns={'RN_DAG_IVMT': 'DAG_IVMT', 'RN_PYN_BNF': 'PYN_BNF'}, inplace=True)
자동차_비례출재_직전3년보험금손조비_가공 = 자동차_비례출재_직전3년보험금손조비_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'CRD_GRD', 'PYN_BNF', 'DAG_IVMT']]

자동차_비비례출재_직전3년보험금손조비_가공 = 자동차_비비례출재_직전3년보험금손조비.copy()
자동차_비비례출재_직전3년보험금손조비_가공['RRNR_DVCD'] = '03'
자동차_비비례출재_직전3년보험금손조비_가공['DMFR_DVCD'] = '01'
자동차_비비례출재_직전3년보험금손조비_가공['BOZ_CD'] = 'B007' # 수정필요
자동차_비비례출재_직전3년보험금손조비_가공['CRD_GRD'] = clsf_crd_grd(자동차_비비례출재_직전3년보험금손조비_가공, 재보험자_국내신용등급)
자동차_비비례출재_직전3년보험금손조비_가공.rename(columns={'WNCR_CNV_DAG_INV_CS': 'DAG_IVMT', 'WNCR_CNV_DS_RBNF_NET': 'PYN_BNF'}, inplace=True)
자동차_비비례출재_직전3년보험금손조비_가공 = 자동차_비비례출재_직전3년보험금손조비_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'CRD_GRD', 'PYN_BNF', 'DAG_IVMT']]

자동차_직전3년보험금손조비_가공 = pd.concat([자동차_원수_직전3년보험금손조비_가공, 자동차_비례출재_직전3년보험금손조비_가공, 자동차_비비례출재_직전3년보험금손조비_가공], axis=0).reset_index(drop=True)

## 개별추산액
자동차_원수_개별추산액_가공 = 자동차_원수_개별추산액.copy()
자동차_원수_개별추산액_가공['RRNR_DVCD'] = '01'
자동차_원수_개별추산액_가공['DMFR_DVCD'] = '01'
자동차_원수_개별추산액_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_원수_개별추산액_가공, 자동차_상품군정보)
자동차_원수_개별추산액_가공['CRD_GRD'] = '#'
자동차_원수_개별추산액_가공.rename(columns={'LTPD_OST_BNF': 'OST_BNF'}, inplace=True)
자동차_원수_개별추산액_가공 = 자동차_원수_개별추산액_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'CRD_GRD', 'OST_BNF']]

자동차_비례출재_개별추산액_가공 = 자동차_비례출재_개별추산액.copy()
자동차_비례출재_개별추산액_가공['RRNR_DVCD'] = '03'
자동차_비례출재_개별추산액_가공['DMFR_DVCD'] = '01'
자동차_비례출재_개별추산액_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_비례출재_개별추산액_가공, 자동차_상품군정보)
자동차_비례출재_개별추산액_가공['T02_RN_RINSC_CD'] = '124012'
자동차_비례출재_개별추산액_가공['CRD_GRD'] = clsf_crd_grd(자동차_비례출재_개별추산액_가공, 재보험자_국내신용등급)
자동차_비례출재_개별추산액_가공.rename(columns={'RN_LTPD_OST_BNF': 'OST_BNF'}, inplace=True)
자동차_비례출재_개별추산액_가공 = 자동차_비례출재_개별추산액_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'CRD_GRD', 'OST_BNF']]

자동차_비비례출재_개별추산액_가공 = 자동차_비비례출재_개별추산액.copy()
자동차_비비례출재_개별추산액_가공['RRNR_DVCD'] = '03'
자동차_비비례출재_개별추산액_가공['DMFR_DVCD'] = '01'
자동차_비비례출재_개별추산액_가공['BOZ_CD'] = 'B007' # 수정필요
자동차_비비례출재_개별추산액_가공['CRD_GRD'] = clsf_crd_grd(자동차_비비례출재_개별추산액_가공, 재보험자_국내신용등급)
자동차_비비례출재_개별추산액_가공.rename(columns={'WNCR_CNV_OST_RBNF': 'OST_BNF'}, inplace=True)
자동차_비비례출재_개별추산액_가공 = 자동차_비비례출재_개별추산액_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'CRD_GRD', 'OST_BNF']]

자동차_개별추산액_가공 = pd.concat([자동차_원수_개별추산액_가공, 자동차_비례출재_개별추산액_가공, 자동차_비비례출재_개별추산액_가공], axis=0).reset_index(drop=True)

In [85]:
# 가정산출
## 최종손해율
# 자동차_최종손해율

## 할인요소
자동차_할인요소 = get_disc_factor_all_c(자동차_보험금진전추이, 할인율.query('KICS_SCEN_NO == 1'))

## 재보험자산 손실조정율
손실조정율 = get_loss_adj_rate_all_c(자동차_보험금진전추이, 할인율.query('KICS_SCEN_NO == 1'), 선도부도율)

## 손해조사비율
자동차_손해조사비율 = 자동차_원수_직전3년보험금손조비_가공 \
    .groupby('BOZ_CD')[['PYN_BNF', 'AT_DAG_IVMT']].sum() \
    .eval('LAE_RATIO = AT_DAG_IVMT/PYN_BNF').reset_index() \
    [['BOZ_CD', 'LAE_RATIO']]
자동차_손해조사비율['RRNR_DVCD'] = '01'
자동차_손해조사비율

KeyError: "Columns not found: 'AT_DAG_IVMT'"

In [65]:
자동차_원수_미경과보험료_가공 \
    .merge(자동차_최종손해율, on=['AT_CHN_DVCD', 'AT_PDGR_CD'], how='left') \
    .merge(자동차_손해조사비율, on=['BOZ_CD', 'RRNR_DVCD'], how='left') \
    .assign(LAE_RATIO = lambda x: x['LAE_RATIO'].fillna(0).astype(float)) \
    .merge(자동차_할인요소[['BOZ_CD', 'DISC_FAC_PRM']], on=['BOZ_CD'], how='left')

,BSC_CVR_CD,PDGR_CD,PDC_CD,INER_CHN_DVCD,USTK_DVCD,LTPD_URND_PRM,AT_PDGR_CD,AT_BSC_CVR_CD,AT_CHN_DVCD,RRNR_DVCD,DMFR_DVCD,BOZ_CD,FNAL_LSRT,LAE_RATIO,DISC_FAC_PRM
0,09,09,20055,14,1,863439937,02,09,04,01,01,B004,1,0,1
1,01,09,20023,01,1,16029630462,02,01,03,01,01,B003,1,0,1
2,02,09,20016,99,1,18762792018,02,01,01,01,01,B003,1,0,1
3,01,08,20014,01,1,66398035275,01,01,03,01,01,B001,1,0,1
4,09,08,20001,04,1,14231290541,01,09,02,01,01,B002,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,03,19,20040,10,2,1200700,11,03,01,01,01,B002,1,0,1
462,01,21,20042,11,2,13363,11,01,01,01,01,B005,1,0,1
463,02,10,20058,10,1,471772,03,01,01,01,01,B005,1,0,1
464,02,20,20041,10,2,462683,11,01,01,01,01,B003,1,0,1
